1. 📦 Configuration et Imports

In [5]:
%pip install mlflow
%pip install xgboost


Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 2.4 MB/s eta 0:00:30
    --------------------------------------- 1.0/72.0 MB 2.4 MB/s eta 0:00:30
    --------------------------------------- 1.6/72.0 MB 2.4 MB/s eta 0:00:30
   - -------------------------------------- 2.1/72.0 MB 2.3 MB/s eta 0:00:30
   - -------------------------------------- 2.4/72.0 MB 2.3 MB/s eta 0:00:31
   - -------------------------------------- 2.9/72.0 MB 2.3 MB/s eta 0:00:31
   - -------------------------------------- 3.4/72.0 MB 2.3 MB/s eta 0:00:30
   -- ------------------------------------- 3.7/72.0 MB 2.2 MB/s eta 0:00:31
   -- ------------------------------------- 4.5/72.0 MB 2.3 MB/s eta 0:00:30
   -- ------------------------------------- 4.7/72.0 MB 2.3 MB/s eta 0:00:30
   -- ------------------

In [7]:
%pip install lightgbm
%pip install catboost

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 2.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.4 MB 2.4 MB/s eta 0:00:43
   ---------------------------------------- 1.0/102.4 MB 1.9 MB/s eta 0:00:55
    --------------------------------------- 1.3/102.4 MB 2.0 MB/s eta 0:00:52
    --------------------------------------- 1.8/102.4 MB 2.1 MB/s eta 0:00:48
    --------------------------------------- 2.4/102.4 MB 2.1 MB/s eta 0:00:47
   - ----------------------------------

In [8]:
# Imports
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
from pathlib import Path

# MLflow
import mlflow
import mlflow.sklearn
from dotenv import load_dotenv

# ML Models
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports terminés")

✅ Imports terminés


In [10]:
# CONFIGURATION MLFLOW LOCAL
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
EXPERIMENT_NAME = "churn_prediction"

# MLflow local
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Désactiver les features incompatibles (sécurité)
os.environ["MLFLOW_ENABLE_LOGGED_MODEL_CREATION"] = "false"

print("✅ MLflow configuré")
print(f"📊 Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"🧪 Experiment: {EXPERIMENT_NAME}")

2025/12/17 13:09:44 INFO mlflow.tracking.fluent: Experiment with name 'churn_prediction' does not exist. Creating a new experiment.


✅ MLflow configuré
📊 Tracking URI: http://127.0.0.1:5000
🧪 Experiment: churn_prediction


2. 📂 Chargement des Données

In [11]:
# Charger les données preprocessées
DATA_PATH = 'processors/preprocessed_data.pkl'

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

print("✅ Données chargées")
print(f"   Train: {X_train.shape}")
print(f"   Test: {X_test.shape}")

✅ Données chargées
   Train: (42070, 40)
   Test: (6000, 40)


3. 🤖 Fonctions Utilitaires

In [12]:
# Fonction pour calculer les métriques
def calculate_metrics(y_true, y_pred, y_proba):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_proba)
    }

# Fonction pour logger un modèle dans MLflow
def log_model_mlflow(model, model_name, stage, metrics, duration, best_params=None):
    """
    Log un modèle dans MLflow
    """
    with mlflow.start_run(run_name=f"{model_name}_{stage}"):
        # Log params
        mlflow.log_param('model_name', model_name)
        mlflow.log_param('stage', stage)
        mlflow.log_param('n_features', X_train.shape[1])
        
        # Log best params si disponibles
        if best_params:
            for k, v in best_params.items():
                try:
                    mlflow.log_param(f'best_{k}', v)
                except:
                    pass
        
        # Log metrics
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        mlflow.log_metric('training_duration', duration)
        
        # Sauvegarder le modèle localement
        model_filename = f"{model_name}_{stage}.pkl"
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)
        
        # Log comme artifact
        try:
            mlflow.log_artifact(model_filename)
        except:
            pass
        
        run_id = mlflow.active_run().info.run_id
        return run_id, model_filename

print("✅ Fonctions utilitaires définies")

✅ Fonctions utilitaires définies


4. 🚀 Entraînement des Modèles Baseline (4 modèles)

In [13]:
# Définir les modèles baseline
baseline_config = {
        'XGBoost': XGBClassifier(
            n_estimators=150,
            max_depth=7,
            learning_rate=0.05,
            subsample=0.85,
            colsample_bytree=0.85,
            min_child_weight=3,
            gamma=0.1,
            random_state=42,
            eval_metric='auc',
            use_label_encoder=False,
            tree_method='hist'
        ),
        
        'LightGBM': LGBMClassifier(
            n_estimators=150,
            max_depth=8,
            learning_rate=0.05,
            num_leaves=40,
            min_child_samples=25,
            subsample=0.85,
            colsample_bytree=0.85,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
            verbose=-1,
            importance_type='gain'
        ),
        
        'Random_Forest': RandomForestClassifier(
            n_estimators=200,
            max_depth=25,
            min_samples_split=15,
            min_samples_leaf=5,
            max_features='sqrt',
            class_weight='balanced_subsample',
            bootstrap=True,
            random_state=42,
            n_jobs=-1,
            warm_start=False
        ),
        
        'CatBoost': CatBoostClassifier(
            iterations=150,
            depth=7,
            learning_rate=0.05,
            l2_leaf_reg=3,
            border_count=128,
            auto_class_weights='Balanced',
            random_state=42,
            verbose=False,
            task_type='CPU',
            bootstrap_type='Bernoulli',
            subsample=0.85
        ),
        
        'Logistic_Regression_ElasticNet': LogisticRegression(
            penalty='elasticnet',
            C=1.0,
            l1_ratio=0.5,
            solver='saga',
            max_iter=1000,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
            warm_start=False
        )
    }

baseline_results = []
trained_models = {}

print("🚀 Entraînement des modèles BASELINE...\n")

for name, model in baseline_config.items():
    print(f"📊 {name}...", end=" ")
    start = datetime.now()
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Métriques
    metrics = calculate_metrics(y_test, y_pred, y_proba)
    duration = (datetime.now() - start).total_seconds()
    
    # Log dans MLflow
    run_id, model_file = log_model_mlflow(model, name, 'baseline', metrics, duration)
    
    # Stocker
    trained_models[f"{name}_baseline"] = model
    baseline_results.append({
        'model': name,
        'stage': 'baseline',
        'run_id': run_id,
        **metrics,
        'duration': duration
    })
    
    print(f"ROC-AUC: {metrics['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Baseline terminé!")

🚀 Entraînement des modèles BASELINE...

📊 XGBoost... 🏃 View run XGBoost_baseline at: http://127.0.0.1:5000/#/experiments/1/runs/fa51421069694b5cbf46b35bd5e7131d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9995 (1.9s)
📊 LightGBM... 🏃 View run LightGBM_baseline at: http://127.0.0.1:5000/#/experiments/1/runs/4fc7aee4be1348799351bde6aa829063
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9995 (1.5s)
📊 Random_Forest... 🏃 View run Random_Forest_baseline at: http://127.0.0.1:5000/#/experiments/1/runs/0ec98cee04fb46a1bc75ed79b2f752db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9971 (7.9s)
📊 CatBoost... 🏃 View run CatBoost_baseline at: http://127.0.0.1:5000/#/experiments/1/runs/418cc395e365448da2f9587792152ddf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9998 (4.6s)
📊 Logistic_Regression_ElasticNet... 🏃 View run Logistic_Regression_ElasticNet_baseline at: http://127.0.0.1:5000/#/experiments/1

5. 🔍 Fine-Tuning (4 modèles avec n_iter=5)

In [14]:
# Grilles de recherche simplifiées
search_spaces = {
        'XGBoost': {
            'n_estimators': randint(100, 500),
            'max_depth': randint(3, 12),
            'learning_rate': uniform(0.01, 0.29),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'gamma': uniform(0, 0.5),
            'min_child_weight': randint(1, 10),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 2)
        },
        
        'LightGBM': {
            'n_estimators': randint(100, 500),
            'max_depth': randint(-1, 15),
            'learning_rate': uniform(0.01, 0.29),
            'num_leaves': randint(15, 150),
            'min_child_samples': randint(10, 60),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 2),
            'min_split_gain': uniform(0, 0.1)
        },
        
        'Random_Forest': {
            'n_estimators': randint(100, 500),
            'max_depth': [15, 20, 25, 30, 35, None],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', 0.5, 0.7, 0.9],
            'max_samples': uniform(0.7, 0.3),
            'class_weight': ['balanced', 'balanced_subsample']
        },
        
        'CatBoost': {
            'iterations': randint(100, 500),
            'depth': randint(4, 11),
            'learning_rate': uniform(0.01, 0.29),
            'l2_leaf_reg': uniform(1, 9),
            'border_count': [32, 64, 128, 200, 254],
            'random_strength': uniform(0, 2)
        },
        
        #Logistic Regression (ElasticNet)
        'Logistic_Regression_ElasticNet': {
            'C': uniform(0.001, 10),
            'l1_ratio': uniform(0, 1),
            'max_iter': randint(500, 2000),
            'tol': uniform(1e-5, 1e-3)
        }
    }

tuned_results = []
N_ITER = 10  # Nombre d'itérations
CV_FOLDS = 3

print(f"🔍 Fine-Tuning ({N_ITER} iterations × {CV_FOLDS} folds)...\n")

for name, base_model in baseline_config.items():
    print(f"📊 {name}...", end=" ")
    start = datetime.now()
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        base_model,
        search_spaces[name],
        n_iter=N_ITER,
        cv=CV_FOLDS,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    search.fit(X_train, y_train)
    best_model = search.best_estimator_
    
    # Prédictions
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    
    # Métriques
    metrics = calculate_metrics(y_test, y_pred, y_proba)
    duration = (datetime.now() - start).total_seconds()
    
    # Log dans MLflow
    run_id, model_file = log_model_mlflow(
        best_model, name, 'tuned', metrics, duration, search.best_params_
    )
    
    # Stocker
    trained_models[f"{name}_tuned"] = best_model
    tuned_results.append({
        'model': name,
        'stage': 'tuned',
        'run_id': run_id,
        **metrics,
        'duration': duration
    })
    
    print(f"ROC-AUC: {metrics['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Fine-tuning terminé!")

🔍 Fine-Tuning (10 iterations × 3 folds)...

📊 XGBoost... 🏃 View run XGBoost_tuned at: http://127.0.0.1:5000/#/experiments/1/runs/c914f0e8f0d94e5db2e98402c4f6e9d6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9997 (43.5s)
📊 LightGBM... 🏃 View run LightGBM_tuned at: http://127.0.0.1:5000/#/experiments/1/runs/310dbd13669947cba9cca7faca443bca
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9997 (40.9s)
📊 Random_Forest... 🏃 View run Random_Forest_tuned at: http://127.0.0.1:5000/#/experiments/1/runs/10e6221a68fd4730802015a3b999a042
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9974 (495.0s)
📊 CatBoost... 🏃 View run CatBoost_tuned at: http://127.0.0.1:5000/#/experiments/1/runs/e379b1fa8a1d45f8bcea14773f56b3fa
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9998 (382.0s)
📊 Logistic_Regression_ElasticNet... 🏃 View run Logistic_Regression_ElasticNet_tuned at: http://127.0.0.1:5000/#/experiments/1/runs

6. 🎯 Stacking Ensembles (4 modèles)

In [15]:
estimators = [
    #('xgb', trained_models['XGBoost_tuned']),
    ('lgbm', trained_models['LightGBM_tuned']),
    ('rf', trained_models['Random_Forest_tuned']),
    ('cat', trained_models['CatBoost_tuned']),
    ('lr', trained_models['Logistic_Regression_ElasticNet_tuned'])
]

ensemble_results = []

meta_learner_config=None

print("🚀 Entraînement des modèles ENSEMBLE...\n")

# 1. Stacking avec Logistic Regression
print("📊 Stacking (LogReg)...", end=" ")

if meta_learner_config is None:
    meta_learner = LogisticRegression(
        penalty='elasticnet',
        C=0.5,
        l1_ratio=0.3,
        solver='saga',
        max_iter=1500,
        random_state=42,
        class_weight='balanced',
        n_jobs=-1
    )
else:
    meta_learner = meta_learner_config

print(f"\n🧠 Meta-Learner (Stacking):")
print(f"   • Type: {type(meta_learner).__name__}")
print(f"   • Configuration: Régularisation ElasticNet")

start = datetime.now()

stacking_lr = StackingClassifier(
        estimators=estimators,
        final_estimator=meta_learner,
        cv=5,
        stack_method='predict_proba',
        n_jobs=-1,
        passthrough=False,
        verbose=0
    )

stacking_lr.fit(X_train, y_train)
y_pred = stacking_lr.predict(X_test)
y_proba = stacking_lr.predict_proba(X_test)[:, 1]

metrics_stack_lr = calculate_metrics(y_test, y_pred, y_proba)
duration = (datetime.now() - start).total_seconds()

run_id_lr, _ = log_model_mlflow(stacking_lr, 'Stacking_LR', 'ensemble', metrics_stack_lr, duration)
trained_models['Stacking_LR'] = stacking_lr
ensemble_results.append({
    'model': 'Stacking_LR',
    'stage': 'ensemble',
    'run_id': run_id_lr,
    **metrics_stack_lr,
    'duration': duration
})

print(f"ROC-AUC: {metrics_stack_lr['roc_auc']:.4f} ({duration:.1f}s)")

# 2. Voting Classifier (soft voting)
print("📊 Voting (Soft)...", end=" ")
start = datetime.now()

voting_soft = VotingClassifier(
        estimators=estimators,
        voting='soft',
        n_jobs=-1,
        flatten_transform=True,
        verbose=False
    )

voting_soft.fit(X_train, y_train)
y_pred = voting_soft.predict(X_test)
y_proba = voting_soft.predict_proba(X_test)[:, 1]

metrics_voting_soft = calculate_metrics(y_test, y_pred, y_proba)
duration = (datetime.now() - start).total_seconds()

run_id_soft, _ = log_model_mlflow(voting_soft, 'Voting_Soft', 'ensemble', metrics_voting_soft, duration)
trained_models['Voting_Soft'] = voting_soft
ensemble_results.append({
    'model': 'Voting_Soft',
    'stage': 'ensemble',
    'run_id': run_id_soft,
    **metrics_voting_soft,
    'duration': duration
})

print(f"ROC-AUC: {metrics_voting_soft['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Ensembles terminés!")

🚀 Entraînement des modèles ENSEMBLE...

📊 Stacking (LogReg)... 
🧠 Meta-Learner (Stacking):
   • Type: LogisticRegression
   • Configuration: Régularisation ElasticNet
🏃 View run Stacking_LR_ensemble at: http://127.0.0.1:5000/#/experiments/1/runs/ec0e7e54492c4a97a953b9b7b0cff8f1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9999 (527.9s)
📊 Voting (Soft)... 🏃 View run Voting_Soft_ensemble at: http://127.0.0.1:5000/#/experiments/1/runs/05031b0c56c84ce5935a5de92d229e17
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
ROC-AUC: 0.9998 (122.9s)

✅ Ensembles terminés!


7. 📊 Lecture des Résultats avec Pandas DataFrame

In [16]:
# Combiner tous les résultats
all_results = baseline_results + tuned_results + ensemble_results
df_results = pd.DataFrame(all_results)

print("📊 Résultats de tous les modèles:\n")
print(df_results[['model', 'stage', 'roc_auc', 'f1_score', 'duration']].to_string(index=False))

# Afficher le top 5 par ROC-AUC
print("\n🏆 Top 5 modèles (ROC-AUC):\n")
top5 = df_results.nlargest(5, 'roc_auc')[['model', 'stage', 'roc_auc', 'f1_score']]
print(top5.to_string(index=False))

📊 Résultats de tous les modèles:

                         model    stage  roc_auc  f1_score   duration
                       XGBoost baseline 0.999541  0.964831   1.874142
                      LightGBM baseline 0.999535  0.966711   1.487259
                 Random_Forest baseline 0.997076  0.917431   7.895275
                      CatBoost baseline 0.999789  0.971504   4.574511
Logistic_Regression_ElasticNet baseline 0.999817  0.948784  56.807263
                       XGBoost    tuned 0.999662  0.973559  43.505387
                      LightGBM    tuned 0.999744  0.972428  40.877317
                 Random_Forest    tuned 0.997352  0.918489 495.023239
                      CatBoost    tuned 0.999765  0.977058 382.041408
Logistic_Regression_ElasticNet    tuned 0.999899  0.954282 367.219170
                   Stacking_LR ensemble 0.999905  0.985953 527.919252
                   Voting_Soft ensemble 0.999842  0.977303 122.855283

🏆 Top 5 modèles (ROC-AUC):

                         mo

In [17]:
# Lire depuis MLflow directement
print("\n📥 Lecture depuis MLflow...\n")

# Obtenir l'ID de l'experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id

# Rechercher toutes les runs
df_mlflow = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.roc_auc > 0",
    order_by=["metrics.roc_auc DESC"]
)

# Afficher les colonnes importantes
if len(df_mlflow) > 0:
    cols_to_show = ['run_id', 'params.model_name', 'params.stage', 
                    'metrics.roc_auc', 'metrics.f1_score', 'metrics.training_duration']
    available_cols = [col for col in cols_to_show if col in df_mlflow.columns]
    print(df_mlflow[available_cols].head(10))
    print(f"\n✅ {len(df_mlflow)} runs trouvées dans MLflow")
else:
    print("⚠️  Aucune run trouvée dans MLflow")


📥 Lecture depuis MLflow...

                             run_id               params.model_name  \
0  ec0e7e54492c4a97a953b9b7b0cff8f1                     Stacking_LR   
1  bc060f83598546fa84afd6a212df8fb3  Logistic_Regression_ElasticNet   
2  05031b0c56c84ce5935a5de92d229e17                     Voting_Soft   
3  c97c335976144b74b9b6880f85e89307  Logistic_Regression_ElasticNet   
4  418cc395e365448da2f9587792152ddf                        CatBoost   
5  e379b1fa8a1d45f8bcea14773f56b3fa                        CatBoost   
6  310dbd13669947cba9cca7faca443bca                        LightGBM   
7  c914f0e8f0d94e5db2e98402c4f6e9d6                         XGBoost   
8  fa51421069694b5cbf46b35bd5e7131d                         XGBoost   
9  4fc7aee4be1348799351bde6aa829063                        LightGBM   

  params.stage  metrics.roc_auc  metrics.f1_score  metrics.training_duration  
0     ensemble         0.999905          0.985953                 527.919252  
1        tuned         0.999899

8. 🏆 Sélection du Meilleur Modèle (ROC-AUC)

In [18]:
# Depuis notre DataFrame local
best_idx = df_results['roc_auc'].idxmax()
best_row = df_results.loc[best_idx]

best_model_name = best_row['model']
best_stage = best_row['stage']
best_run_id = best_row['run_id']
best_roc_auc = best_row['roc_auc']

print("🏆 MEILLEUR MODÈLE (ROC-AUC)")
print("="*60)
print(f"Modèle:    {best_model_name}")
print(f"Stage:     {best_stage}")
print(f"ROC-AUC:   {best_roc_auc:.4f}")
print(f"F1-Score:  {best_row['f1_score']:.4f}")
print(f"Precision: {best_row['precision']:.4f}")
print(f"Recall:    {best_row['recall']:.4f}")
print(f"Run ID:    {best_run_id}")
print("="*60)

# Récupérer le modèle
best_model_key = f"{best_model_name}_{best_stage}" if best_stage != 'ensemble' else best_model_name
best_model = trained_models.get(best_model_key)

print(f"\n✅ Modèle chargé : {type(best_model).__name__}")

🏆 MEILLEUR MODÈLE (ROC-AUC)
Modèle:    Stacking_LR
Stage:     ensemble
ROC-AUC:   0.9999
F1-Score:  0.9860
Precision: 0.9775
Recall:    0.9946
Run ID:    ec0e7e54492c4a97a953b9b7b0cff8f1

✅ Modèle chargé : StackingClassifier


9. 🔄 Chargement du Modèle depuis MLflow

In [19]:
# Charger le modèle depuis le run_id
print(f"📥 Chargement du modèle depuis run_id: {best_run_id}\n")

# Via artifact (compatible DagsHub)
try:
    # Télécharger l'artifact
    model_filename = f"{best_model_name}_{best_stage}.pkl"
    artifact_uri = f"runs:/{best_run_id}/{model_filename}"
    
    local_path = mlflow.artifacts.download_artifacts(artifact_uri)
    
    # Charger avec pickle
    with open(local_path, 'rb') as f:
        loaded_model = pickle.load(f)
    
    print(f"✅ Modèle chargé depuis MLflow artifact")
    print(f"   Type: {type(loaded_model).__name__}")
    
except Exception as e:
    print(f"⚠️  Erreur de chargement depuis MLflow: {e}")
    print(f"   Utilisation du modèle en mémoire à la place")
    loaded_model = best_model

# Alternative: Charger depuis fichier local
local_model_file = f"{best_model_name}_{best_stage}.pkl"
if os.path.exists(local_model_file):
    with open(local_model_file, 'rb') as f:
        loaded_model_local = pickle.load(f)
    print(f"✅ Modèle également disponible localement: {local_model_file}")

📥 Chargement du modèle depuis run_id: ec0e7e54492c4a97a953b9b7b0cff8f1



✅ Modèle chargé depuis MLflow artifact
   Type: StackingClassifier
✅ Modèle également disponible localement: Stacking_LR_ensemble.pkl


In [20]:
# Test du modèle chargé
print("\n🧪 Test du modèle chargé...\n")

# Prédictions
y_pred_loaded = loaded_model.predict(X_test)
y_proba_loaded = loaded_model.predict_proba(X_test)[:, 1]

# Métriques
test_metrics = calculate_metrics(y_test, y_pred_loaded, y_proba_loaded)

print("📊 Performances du modèle chargé:")
for metric, value in test_metrics.items():
    print(f"   {metric:12s}: {value:.4f}")

# Test sur quelques exemples
print("\n🔍 Prédictions sur 5 exemples:")
sample_predictions = loaded_model.predict(X_test[:5])
sample_probas = loaded_model.predict_proba(X_test[:5])[:, 1]

for i in range(5):
    print(f"   Sample {i+1}: Churn={sample_predictions[i]}, Proba={sample_probas[i]:.4f}")

print("\n✅ Modèle fonctionne correctement!")


🧪 Test du modèle chargé...

📊 Performances du modèle chargé:
   accuracy    : 0.9965
   precision   : 0.9775
   recall      : 0.9946
   f1_score    : 0.9860
   roc_auc     : 0.9999

🔍 Prédictions sur 5 exemples:
   Sample 1: Churn=0, Proba=0.0000
   Sample 2: Churn=0, Proba=0.0000
   Sample 3: Churn=0, Proba=0.0000
   Sample 4: Churn=0, Proba=0.0000
   Sample 5: Churn=0, Proba=0.0090

✅ Modèle fonctionne correctement!


In [21]:


print("\n" + "="*80)
print("📦 ENREGISTREMENT DANS MLFLOW MODEL REGISTRY")
print("="*80)

# 1. Enregistrer le modèle dans MLflow Model Registry
model_name = f"churn_prediction_{best_model_name}"

print(f"\n🔄 Enregistrement du modèle: {model_name}")
print(f"   Run ID: {best_run_id}")

try:
    # Log le modèle avec mlflow.sklearn
    with mlflow.start_run(run_id=best_run_id):
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="model",
            registered_model_name=model_name
        )
    
    print(f"✅ Modèle enregistré dans MLflow Model Registry")
    print(f"   Nom: {model_name}")
    
    # 2. Obtenir la dernière version
    client = mlflow.tracking.MlflowClient()
    latest_versions = client.get_latest_versions(model_name, stages=["None"])
    
    if latest_versions:
        latest_version = latest_versions[0].version
        print(f"   Version: {latest_version}")
        
        # 3. Transitionner vers Production
        print(f"\n🚀 Transition vers Production...")
        client.transition_model_version_stage(
            name=model_name,
            version=latest_version,
            stage="Production",
            archive_existing_versions=True
        )
        
        print(f"✅ Modèle mis en Production")
        print(f"   Stage: Production")
        print(f"   Version: {latest_version}")
        
        # 4. Ajouter une description
        client.update_model_version(
            name=model_name,
            version=latest_version,
            description=f"""
            Best performing churn prediction model
            - Model: {best_model_name}
            - Stage: {best_stage}
            - ROC-AUC: {best_roc_auc:.4f}
            - F1-Score: {best_row['f1_score']:.4f}
            - Trained: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            """
        )
        
        # 5. Ajouter des tags
        client.set_model_version_tag(
            name=model_name,
            version=latest_version,
            key="model_type",
            value=best_model_name
        )
        
        client.set_model_version_tag(
            name=model_name,
            version=latest_version,
            key="training_stage",
            value=best_stage
        )
        
        client.set_model_version_tag(
            name=model_name,
            version=latest_version,
            key="roc_auc",
            value=str(round(best_roc_auc, 4))
        )
        
        print(f"✅ Métadonnées ajoutées (description + tags)")
        
except Exception as e:
    print(f"⚠️  Erreur lors de l'enregistrement: {e}")
    print(f"   Le modèle reste disponible via run_id: {best_run_id}")

# 6. Fonction pour charger depuis MLflow Model Registry
def load_model_from_registry(model_name, stage="Production"):
    """
    Charge un modèle depuis MLflow Model Registry
    
    Args:
        model_name: Nom du modèle dans le registry
        stage: Stage du modèle (Production, Staging, Archived, None)
    
    Returns:
        model: Le modèle chargé
    """
    model_uri = f"models:/{model_name}/{stage}"
    model = mlflow.sklearn.load_model(model_uri)
    return model

# 7. Test de chargement depuis MLflow Registry
print("\n🔄 Test de chargement depuis MLflow Model Registry...\n")

try:
    loaded_model_mlflow = load_model_from_registry(model_name, stage="Production")
    
    # Test de prédiction
    y_pred_test = loaded_model_mlflow.predict(X_test[:5])
    y_proba_test = loaded_model_mlflow.predict_proba(X_test)[:, 1]
    
    # Calculer les métriques
    mlflow_metrics = calculate_metrics(y_test, loaded_model_mlflow.predict(X_test), y_proba_test)
    
    print(f"✅ Modèle chargé depuis MLflow Registry")
    print(f"   URI: models:/{model_name}/Production")
    print(f"   Type: {type(loaded_model_mlflow).__name__}")
    print(f"\n📊 Performances:")
    for metric, value in mlflow_metrics.items():
        print(f"   {metric:12s}: {value:.4f}")
    
    print(f"\n🧪 Test sur 5 exemples:")
    for i in range(5):
        print(f"   Sample {i+1}: Prediction={y_pred_test[i]}")
    
    print("\n✅ Le modèle fonctionne correctement!")
    
except Exception as e:
    print(f"⚠️  Erreur de chargement: {e}")

# 8. Afficher toutes les versions du modèle
print("\n📋 Versions du modèle dans le Registry:\n")

try:
    client = mlflow.tracking.MlflowClient()
    
    # Récupérer toutes les versions
    all_versions = client.search_model_versions(f"name='{model_name}'")
    
    if all_versions:
        print(f"{'Version':<10} {'Stage':<15} {'Created':<20} {'Run ID':<40}")
        print("-" * 90)
        
        for mv in all_versions:
            created = datetime.fromtimestamp(mv.creation_timestamp/1000).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{mv.version:<10} {mv.current_stage:<15} {created:<20} {mv.run_id:<40}")
        
        print(f"\n✅ {len(all_versions)} version(s) trouvée(s)")
    else:
        print("⚠️  Aucune version trouvée")
        
except Exception as e:
    print(f"⚠️  Erreur: {e}")

# 9. Exemple d'utilisation en production
print("\n" + "="*80)
print("💡 UTILISATION EN PRODUCTION")
print("="*80)

print(f"""
# Charger le modèle en production:
import mlflow

model = mlflow.sklearn.load_model("models:/{model_name}/Production")

# Faire des prédictions:
predictions = model.predict(X_new)
probabilities = model.predict_proba(X_new)[:, 1]

# Ou charger une version spécifique:
model_v1 = mlflow.sklearn.load_model("models:/{model_name}/1")

# Ou charger depuis un run_id:
model_from_run = mlflow.sklearn.load_model("runs:/{best_run_id}/model")
""")

print("\n" + "="*80)
print("✅ ENREGISTREMENT MLFLOW MODEL REGISTRY TERMINÉ")
print("="*80)

2025/12/17 13:49:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



📦 ENREGISTREMENT DANS MLFLOW MODEL REGISTRY

🔄 Enregistrement du modèle: churn_prediction_Stacking_LR
   Run ID: ec0e7e54492c4a97a953b9b7b0cff8f1


Successfully registered model 'churn_prediction_Stacking_LR'.
2025/12/17 13:50:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: churn_prediction_Stacking_LR, version 1
Created version '1' of model 'churn_prediction_Stacking_LR'.


🏃 View run Stacking_LR_ensemble at: http://127.0.0.1:5000/#/experiments/1/runs/ec0e7e54492c4a97a953b9b7b0cff8f1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
✅ Modèle enregistré dans MLflow Model Registry
   Nom: churn_prediction_Stacking_LR
   Version: 1

🚀 Transition vers Production...
✅ Modèle mis en Production
   Stage: Production
   Version: 1
✅ Métadonnées ajoutées (description + tags)

🔄 Test de chargement depuis MLflow Model Registry...



✅ Modèle chargé depuis MLflow Registry
   URI: models:/churn_prediction_Stacking_LR/Production
   Type: StackingClassifier

📊 Performances:
   accuracy    : 0.9965
   precision   : 0.9775
   recall      : 0.9946
   f1_score    : 0.9860
   roc_auc     : 0.9999

🧪 Test sur 5 exemples:
   Sample 1: Prediction=0
   Sample 2: Prediction=0
   Sample 3: Prediction=0
   Sample 4: Prediction=0
   Sample 5: Prediction=0

✅ Le modèle fonctionne correctement!

📋 Versions du modèle dans le Registry:

Version    Stage           Created              Run ID                                  
------------------------------------------------------------------------------------------
1          Production      2025-12-17 13:50:17  ec0e7e54492c4a97a953b9b7b0cff8f1        

✅ 1 version(s) trouvée(s)

💡 UTILISATION EN PRODUCTION

# Charger le modèle en production:
import mlflow

model = mlflow.sklearn.load_model("models:/churn_prediction_Stacking_LR/Production")

# Faire des prédictions:
predictions = model.

10. 📦 Enregistrement dans Model Registry (Local)

In [25]:
# Créer un Model Registry local
MODEL_REGISTRY_DIR = Path("processors/model_registry")
MODEL_REGISTRY_DIR.mkdir(exist_ok=True)

def register_model(model, model_name, version="1.0.0", stage="production"):
    """
    Enregistre un modèle dans le registry local
    """
    import json
    import shutil
    
    # Créer la structure
    model_dir = MODEL_REGISTRY_DIR / model_name.replace(" ", "_")
    model_dir.mkdir(exist_ok=True)
    
    version_dir = model_dir / version
    version_dir.mkdir(exist_ok=True)
    
    # Sauvegarder le modèle
    model_path = version_dir / "model.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # Métadonnées
    metadata = {
        "model_name": model_name,
        "version": version,
        "stage": stage,
        "registered_at": datetime.now().isoformat(),
        "metrics": test_metrics,
        "run_id": best_run_id
    }
    
    with open(version_dir / "metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    # Lien production
    if stage == "production":
        prod_path = model_dir / "production.pkl"
        shutil.copy(model_path, prod_path)
    
    return str(model_path)

# Enregistrer le meilleur modèle
print("📦 Enregistrement dans Model Registry...\n")

registry_name = f"Best_Churn_{best_model_name}"
model_path = register_model(
    model=loaded_model,
    model_name=registry_name,
    version="1.0.0",
    stage="production"
)

print(f"✅ Modèle enregistré dans le registry")
print(f"   Nom: {registry_name}")
print(f"   Version: 1.0.0")
print(f"   Stage: production")
print(f"   Path: {model_path}")

📦 Enregistrement dans Model Registry...

✅ Modèle enregistré dans le registry
   Nom: Best_Churn_Stacking_LR
   Version: 1.0.0
   Stage: production
   Path: processors\model_registry\Best_Churn_Stacking_LR\1.0.0\model.pkl


In [26]:
# Fonction pour charger depuis le registry
def load_from_registry(model_name, stage="production"):
    """Charge un modèle depuis le registry local"""
    import json
    
    model_dir = MODEL_REGISTRY_DIR / model_name.replace(" ", "_")
    model_path = model_dir / f"{stage}.pkl"
    
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # Charger les métadonnées
    versions = [d for d in model_dir.iterdir() if d.is_dir()]
    if versions:
        latest_version = sorted(versions)[-1]
        with open(latest_version / "metadata.json", 'r') as f:
            metadata = json.load(f)
    else:
        metadata = {}
    
    return model, metadata

# Test du chargement
print("\n🔄 Test de chargement depuis le registry...\n")

loaded_from_registry, metadata = load_from_registry(registry_name, stage="production")

print(f"✅ Modèle chargé depuis le registry")
print(f"   Nom: {metadata.get('model_name', 'N/A')}")
print(f"   Version: {metadata.get('version', 'N/A')}")
print(f"   ROC-AUC: {metadata.get('metrics', {}).get('roc_auc', 0):.4f}")

# Test de prédiction
test_pred = loaded_from_registry.predict(X_test[:5])
print(f"\n🧪 Test de prédiction: {test_pred}")
print("✅ Le modèle fonctionne correctement!")


🔄 Test de chargement depuis le registry...

✅ Modèle chargé depuis le registry
   Nom: Best_Churn_Stacking_LR
   Version: 1.0.0
   ROC-AUC: 0.9999

🧪 Test de prédiction: [0 0 0 0 0]
✅ Le modèle fonctionne correctement!


11. 📊 Résumé Final

In [28]:
print("\n" + "="*80)
print("🎉 RÉSUMÉ FINAL - MLflow Tracking")
print("="*80)

print(f"\n📊 Modèles entraînés:")
print(f"   • Baseline:  5 modèles")
print(f"   • Tuned:     5 modèles (n_iter={N_ITER})")
print(f"   • Ensemble:  2 modèles (Stacking + Voting)")
print(f"   • TOTAL:     12 modèles")

print(f"\n🏆 Meilleur modèle:")
print(f"   • Nom:       {best_model_name}")
print(f"   • Stage:     {best_stage}")
print(f"   • ROC-AUC:   {best_roc_auc:.4f}")
print(f"   • F1-Score:  {best_row['f1_score']:.4f}")

print(f"\n🔗 MLflow:")
print(f"   • Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"   • Experiment:   {EXPERIMENT_NAME}")
print(f"   • Runs totales: {len(df_results)}")

print(f"\n📦 Model Registry:")
print(f"   • Nom:     {registry_name}")
print(f"   • Version: 1.0.0")
print(f"   • Stage:   production")
print(f"   • Path:    {MODEL_REGISTRY_DIR / registry_name.replace(' ', '_')}")

print("\n" + "="*80)
print("✅ Pipeline MLflow terminé avec succès!")
print("="*80)

print("\n💡 Prochaines étapes:")
print("   1. Consultez MLflow UI pour voir toutes les runs")
print("   2. Chargez le modèle avec: load_from_registry()")
print("   3. Déployez en production")
print("   4. Configurez le monitoring")


🎉 RÉSUMÉ FINAL - MLflow Tracking

📊 Modèles entraînés:
   • Baseline:  5 modèles
   • Tuned:     5 modèles (n_iter=10)
   • Ensemble:  2 modèles (Stacking + Voting)
   • TOTAL:     12 modèles

🏆 Meilleur modèle:
   • Nom:       Stacking_LR
   • Stage:     ensemble
   • ROC-AUC:   0.9999
   • F1-Score:  0.9860

🔗 MLflow:
   • Tracking URI: http://127.0.0.1:5000
   • Experiment:   churn_prediction
   • Runs totales: 12

📦 Model Registry:
   • Nom:     Best_Churn_Stacking_LR
   • Version: 1.0.0
   • Stage:   production
   • Path:    processors\model_registry\Best_Churn_Stacking_LR

✅ Pipeline MLflow terminé avec succès!

💡 Prochaines étapes:
   1. Consultez MLflow UI pour voir toutes les runs
   2. Chargez le modèle avec: load_from_registry()
   3. Déployez en production
   4. Configurez le monitoring
